<a href="https://colab.research.google.com/github/TejasvDua/CV/blob/main/resnet4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 1️⃣ Imports and Setup

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet


In [6]:
# Upload kaggle.json
from google.colab import files
files.upload()  # Upload kaggle.json here
# Setup Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Download Dataset
!kaggle datasets download -d shubhamgoel27/dermnet
# Unzip Dataset
!unzip -q dermnet.zip -d dermnet
# Check files
import os
print(os.listdir("/content/dermnet"))
TRAIN_PATH = "/content/dermnet/train"
TEST_PATH = "/content/dermnet/test"


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
['train', 'test']


In [7]:
# 2️⃣ Data Loading with Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE_RESNET = (224, 224)
IMAGE_SIZE_INCEPTION = (299, 299)
BATCH_SIZE = 32
SEED = 42

# Data generators
train_datagen_resnet = ImageDataGenerator(
    preprocessing_function=preprocess_resnet,
    shear_range=0.2,
    zoom_range=0.3,
    rotation_range=30,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

train_datagen_inception = ImageDataGenerator(
    preprocessing_function=preprocess_inception,
    shear_range=0.2,
    zoom_range=0.3,
    rotation_range=30,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

# Paths
TRAIN_PATH = "/content/dermnet/train"
TEST_PATH = "/content/dermnet/test"

# RESNET50 GENERATORS
train_generator_resnet = train_datagen_resnet.flow_from_directory(
    TRAIN_PATH,
    target_size=IMAGE_SIZE_RESNET,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=SEED
)

val_generator_resnet = train_datagen_resnet.flow_from_directory(
    TRAIN_PATH,
    target_size=IMAGE_SIZE_RESNET,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=SEED
)

# INCEPTIONV3 GENERATORS
train_generator_inception = train_datagen_inception.flow_from_directory(
    TRAIN_PATH,
    target_size=IMAGE_SIZE_INCEPTION,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=SEED
)

val_generator_inception = train_datagen_inception.flow_from_directory(
    TRAIN_PATH,
    target_size=IMAGE_SIZE_INCEPTION,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=SEED
)


Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.
Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.


In [8]:
# Test generators
test_datagen_resnet = ImageDataGenerator(preprocessing_function=preprocess_resnet)
test_datagen_inception = ImageDataGenerator(preprocessing_function=preprocess_inception)

test_generator_resnet = test_datagen_resnet.flow_from_directory(
    TEST_PATH,
    target_size=IMAGE_SIZE_RESNET,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator_inception = test_datagen_inception.flow_from_directory(
    TEST_PATH,
    target_size=IMAGE_SIZE_INCEPTION,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)



Found 4002 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [9]:
# 3️⃣ Model Creation

def build_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

num_classes = train_generator_resnet.num_classes

# ResNet50
resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in resnet_base.layers:
    layer.trainable = False
resnet_model = build_model(resnet_base, num_classes)

# InceptionV3
inception_base = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))
for layer in inception_base.layers:
    layer.trainable = False
inception_model = build_model(inception_base, num_classes)



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:

# 4️⃣ Compile and Callbacks

resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
callbacks = [early_stop, reduce_lr]

In [11]:
# 5️⃣ Compute Class Weights

import numpy as np

y_train_resnet = train_generator_resnet.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_resnet), y=y_train_resnet)
class_weights = dict(enumerate(class_weights))


In [2]:
import numpy as np
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator_resnet.classes),
    y=train_generator_resnet.classes
)
class_weights = dict(enumerate(class_weights))

NameError: name 'train_generator_resnet' is not defined

In [8]:
# 6️⃣ Training

history_resnet = resnet_model.fit(
    train_generator_resnet,
    validation_data=val_generator_resnet,
    epochs=25,
    callbacks=callbacks,
    class_weight=class_weights
)

history_inception = inception_model.fit(
    train_generator_inception,
    validation_data=val_generator_inception,
    epochs=25,
    callbacks=callbacks,
    class_weight=class_weights
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 240s 577ms/step - accuracy: 0.1249 - loss: 3.8197 - val_accuracy: 0.1910 - val_loss: 3.0137 - learning_rate: 0.0010
Epoch 2/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 220s 563ms/step - accuracy: 0.2301 - loss: 2.8734 - val_accuracy: 0.2268 - val_loss: 2.8529 - learning_rate: 0.0010
Epoch 3/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 215s 552ms/step - accuracy: 0.2758 - loss: 2.6400 - val_accuracy: 0.2068 - val_loss: 2.9809 - learning_rate: 0.0010
Epoch 4/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 216s 555ms/step - accuracy: 0.2785 - loss: 2.6486 - val_accuracy: 0.2332 - val_loss: 2.8374 - learning_rate: 0.0010
Epoch 5/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 216s 554ms/step - accuracy: 0.2809 - loss: 2.6187 - val_accuracy: 0.2097 - val_loss: 2.8975 - learning_rate: 0.0010
Epoch 6/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 217s 557ms/step - accuracy: 0.2889 - loss: 2.5289 - val_accuracy: 0.1981 - val_loss: 3.0559 - learning_rate: 0.0010
Epoch 7/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 214s 548ms/step - accura

KeyboardInterrupt: 

In [10]:
history2 = resnet_model.fit(
    train_generator_resnet,
    validation_data=val_generator_resnet,
    initial_epoch=16,
    epochs=25,
    callbacks=[early_stop, reduce_lr],  # if defined
    class_weight=class_weights  # if defined
)



Epoch 17/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 221s 566ms/step - accuracy: 0.3296 - loss: 2.2478 - val_accuracy: 0.2516 - val_loss: 2.7371 - learning_rate: 2.5000e-04
Epoch 18/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 216s 553ms/step - accuracy: 0.3343 - loss: 2.2212 - val_accuracy: 0.2542 - val_loss: 2.7055 - learning_rate: 2.5000e-04
Epoch 19/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 216s 554ms/step - accuracy: 0.3369 - loss: 2.2043 - val_accuracy: 0.2345 - val_loss: 2.7858 - learning_rate: 2.5000e-04
Epoch 20/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 217s 557ms/step - accuracy: 0.3388 - loss: 2.1687 - val_accuracy: 0.2374 - val_loss: 2.7493 - learning_rate: 2.5000e-04
Epoch 21/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 216s 554ms/step - accuracy: 0.3489 - loss: 2.1185 - val_accuracy: 0.2552 - val_loss: 2.7080 - learning_rate: 2.5000e-04
Epoch 22/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 214s 549ms/step - accuracy: 0.3415 - loss: 2.1559 - val_accuracy: 0.2526 - val_loss: 2.7282 - learning_rate: 1.2500e-04
Epoch 23/25
390/390 ━━━━━━━━━━━━━━

In [11]:
history2 = inception_model.fit(
    train_generator_inception,
    validation_data=val_generator_inception,
    initial_epoch=1,
    epochs=25,
    callbacks=[early_stop, reduce_lr],  # if using callbacks
    class_weight=class_weights
)


Epoch 2/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 360s 924ms/step - accuracy: 0.2087 - loss: 2.8116 - val_accuracy: 0.2200 - val_loss: 2.7498 - learning_rate: 0.0010
Epoch 3/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 350s 897ms/step - accuracy: 0.2395 - loss: 2.6678 - val_accuracy: 0.2162 - val_loss: 2.7399 - learning_rate: 0.0010
Epoch 4/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 354s 908ms/step - accuracy: 0.2609 - loss: 2.5843 - val_accuracy: 0.2091 - val_loss: 2.8972 - learning_rate: 0.0010
Epoch 5/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 349s 896ms/step - accuracy: 0.2586 - loss: 2.5777 - val_accuracy: 0.1849 - val_loss: 2.9340 - learning_rate: 0.0010
Epoch 6/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 387s 910ms/step - accuracy: 0.2606 - loss: 2.5856 - val_accuracy: 0.2194 - val_loss: 2.7733 - learning_rate: 0.0010
Epoch 7/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 351s 901ms/step - accuracy: 0.2750 - loss: 2.4922 - val_accuracy: 0.2104 - val_loss: 2.8732 - learning_rate: 5.0000e-04
Epoch 8/25
390/390 ━━━━━━━━━━━━━━━━━━━━ 353s 906ms/step - ac

In [17]:
# 7️⃣ Evaluation and Metrics

# Evaluate models
resnet_loss, resnet_acc = resnet_model.evaluate(test_generator_resnet)
inception_loss, inception_acc = inception_model.evaluate(test_generator_inception)

print(f'ResNet50 Test Accuracy: {resnet_acc:.4f}, Loss: {resnet_loss:.4f}')
print(f'InceptionV3 Test Accuracy: {inception_acc:.4f}, Loss: {inception_loss:.4f}')

# Classification report for ResNet
y_true = test_generator_resnet.classes
y_pred = np.argmax(resnet_model.predict(test_generator_resnet), axis=1)
print("ResNet50 Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_generator_resnet.class_indices.keys())))




/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 32s 181ms/step - accuracy: 0.0245 - loss: 4.2978
126/126 ━━━━━━━━━━━━━━━━━━━━ 45s 226ms/step - accuracy: 0.0254 - loss: 3.4251
ResNet50 Test Accuracy: 0.0412, Loss: 4.2193
InceptionV3 Test Accuracy: 0.0532, Loss: 3.2316
126/126 ━━━━━━━━━━━━━━━━━━━━ 26s 176ms/step
ResNet50 Classification Report:
                                                                    precision    recall  f1-score   support

                                           Acne and Rosacea Photos       0.00      0.00      0.00       312
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions       0.25      0.01      0.02       288
                                          Atopic Dermatitis Photos       0.02      0.02      0.02       123
                                            Bullous Disease Photos       0.02      0.04      0.03       113
                Cellulitis Impetigo and other Bacterial Infections       0.00      0.00      0.00        73
                          

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet

def predict(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_resnet(img_array)  # ✅ FIXED!

    preds = model.predict(img_array)
    pred_index = np.argmax(preds)
    pred_class = CLASS_NAMES[pred_index]

    return pred_class


In [22]:
# ✅ Step 1: Install Gradio
!pip install -q gradio

# ✅ Step 2: Gradio app using your ResNet model
import gradio as gr
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Use trained model (make sure resnet_model is already trained above)
model = resnet_model

# Get class labels from the test generator (already defined earlier)
CLASS_NAMES = list(test_generator_resnet.class_indices.keys())

# Make sure labels are extracted from your test set
labels = list(train_generator_resnet.class_indices.keys())
CLASS_NAMES = labels

def predict(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_resnet(img_array)

    preds = resnet_model.predict(img_array)
    pred_index = np.argmax(preds)
    pred_class = CLASS_NAMES[pred_index]
    return pred_class

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="label",
    title="🧠 Skin Disease Classifier",
    description="Upload a skin image to detect disease using a trained ResNet50 model."
)

interface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.1 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ca14eb50d15978c401.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [26]:
from collections import Counter
print(Counter(train_generator_resnet.classes))



Counter({np.int32(14): 1124, np.int32(16): 1097, np.int32(18): 1040, np.int32(5): 988, np.int32(1): 920, np.int32(22): 869, np.int32(12): 832, np.int32(0): 672, np.int32(17): 485, np.int32(9): 455, np.int32(2): 392, np.int32(20): 386, np.int32(11): 371, np.int32(3): 359, np.int32(15): 345, np.int32(10): 336, np.int32(21): 333, np.int32(6): 324, np.int32(8): 324, np.int32(4): 231, np.int32(13): 208, np.int32(7): 192, np.int32(19): 170})
